In [ ]:
# Notebook to duplicate ARGOG using cells 
## code is from main.py

In [ ]:
# !pip install --upgrade llama-index
# !pip show llama-index

# !pip install llama-index-agent-openai  llama-index-cli  llama-index-core  llama-index-embeddings-openai  llama-index-indices-managed-llama-cloud  llama-index-legacy  llama-index-llms-openai  llama-index-multi-modal-llms-openai  llama-index-program-openai  llama-index-question-gen-openai  llama-index-readers-file  llama-index-readers-llama-parse  nltk
# !pip install -U llama-index
# %pip install -q cohere llama-index-postprocessor-cohere-rerank
# !pip install tonic-validate

In [1]:
# importing stuff
import openai
import chromadb
from dotenv import load_dotenv
import pandas as pd
from utils import run_experiment, load_config
from llama_index.llms.openai import OpenAI
from llama_index.core import VectorStoreIndex, PromptTemplate, ServiceContext, Settings
# from llama_index.settings import Settings
from llama_index.embeddings.openai import OpenAIEmbedding
from llama_index.postprocessor.cohere_rerank import CohereRerank
from llama_index.core.indices.query.query_transform import HyDEQueryTransform
from llama_index.core.query_engine import TransformQueryEngine, RetrieverQueryEngine
from llama_index.vector_stores.chroma import ChromaVectorStore
from llama_index.core.postprocessor import LLMRerank
from llama_index.core.retrievers import QueryFusionRetriever
from tonic_validate import ValidateScorer, ValidateApi
from tonic_validate.metrics import RetrievalPrecisionMetric, AnswerSimilarityMetric
import os

In [ ]:
# # sample code to use tonic validate
# from tonic_validate import Benchmark, ValidateApi, ValidateScorer

# # Function to simulate getting a response and context from your LLM
# # Replace this with your actual function call
# def get_rag_response(question):
#     return {
#         "llm_answer": "Paris",
#         "llm_context_list": ["Paris is the capital of France."]
#     }

# benchmark = Benchmark(questions=["What is the capital of France?"], answers=["Paris"])
# # Score the responses for each question and answer pair
# scorer = ValidateScorer()
# run = scorer.score(benchmark, get_rag_response)
# validate_api = ValidateApi("opycxzKWmtHosC25-r_Kd31EbfLHVBerNEBj675AeS4")
# validate_api.upload_run("f6cd8c02-2ef3-42ac-bc69-6c2fd5e3df03", run)

In [ ]:
### SETUP --------------------------------------------------------------------------------------------------------------
# Load the config file (.env vars)
load_config()
# load_dotenv(override=True)
# Set the OpenAI API key for authentication.
openai.api_key = os.getenv("OPENAI_API_KEY")
tonic_validate_api_key = os.getenv("HASSAN_TONIC_VALIDATE_API1")
tonic_validate_project_key = os.getenv("HASSAN_TONIC_VALIDATE_PROJECT1")
tonic_validate_benchmark_key = os.getenv("HASSAN_TONIC_VALIDATE_BENCHMARK1")
validate_api = ValidateApi(tonic_validate_api_key)
# print (f'{tonic_validate_api_key} {tonic_validate_project_key} {tonic_validate_benchmark_key}')
cohere_api_key = os.getenv("HASSAN_COHERE_DEFAULT_API")
# print (f'{tonic_validate_api_key} {tonic_validate_project_key} {tonic_validate_benchmark_key}')
# cohere_api_key



In [ ]:
# load config into dictionary
from dotenv import load_dotenv
import os

# Load the .env file
load_dotenv()

# Create a config dictionary
config = {
    # 'cohere_api_key': os.getenv('COHERE_API_KEY'),
    # 'another_setting': os.getenv('ANOTHER_SETTING'),
    'tonic_validate_api_key': os.getenv("HASSAN_TONIC_VALIDATE_API1"),
    'tonic_validate_project_key':  os.getenv("HASSAN_TONIC_VALIDATE_PROJECT1"),
    'tonic_validate_benchmark_key':  os.getenv("HASSAN_TONIC_VALIDATE_BENCHMARK1"),
    'cohere_api_key':  os.getenv("HASSAN_COHERE_DEFAULT_API")
}

# Print to verify
print(config)


In [3]:
# Service context
llm = OpenAI(model="gpt-3.5-turbo", temperature=0.0)
embed_model = OpenAIEmbedding(model="text-embedding-3-large")
from llama_index.core import Settings

Settings.embed_model = OpenAIEmbedding(model="text-embedding-3-large")
# service_context = ServiceContext.from_defaults(llm = llm, embed_model = embed_model)
# settings = Settings(llm=llm, embed_model=embed_model)
Settings.llm = llm 

chroma_client = chromadb.PersistentClient(path="./chroma_db")

In [ ]:
# # existing_collections = chroma_client.list_collections()
# # print(existing_collections)
# # First, check if collection exists and create it if it doesn't
# try:
#     chroma_collection = chroma_client.get_collection("ai_arxiv_full")
# except:
#     # Create new collection
#     chroma_collection = chroma_client.create_collection(
#         name="ai_arxiv_full",
#         metadata={"description": "ArXiv AI papers collection"}
#     )

# # Now you can use the collection
# vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
# index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [33]:
# from llama_index.core import Settings, StorageContext, VectorStoreIndex
# from llama_index.vector_stores.chroma import ChromaVectorStore
# from llama_index.core.node_parser import SentenceWindowNodeParser  # Updated import path

# # Create the collection first
# try:
#     chroma_collection_sentence_window = chroma_client.get_collection("ai_arxiv_sentence_window")
# except:
#     chroma_collection_sentence_window = chroma_client.create_collection(
#         name="ai_arxiv_sentence_window",
#         metadata={"description": "ArXiv AI papers with sentence window processing"}
#     )

# # Set up sentence window processing
# node_parser = SentenceWindowNodeParser.from_defaults(
#     window_size=3,
#     window_metadata_key="window",
#     original_text_metadata_key="original_text"
# )

# # Update settings with the node parser
# Settings.node_parser = node_parser

# # Set up vector store and index
# vector_store_sentence_window = ChromaVectorStore(
#     chroma_collection=chroma_collection_sentence_window
# )
# storage_context = StorageContext.from_defaults(
#     vector_store=vector_store_sentence_window
# )

# # Create the index
# index_sentence_window = VectorStoreIndex.from_vector_store(
#     vector_store=vector_store_sentence_window
# )

In [4]:
# Traditional VDB
chroma_collection = chroma_client.get_collection("ai_arxiv_full")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)
index = VectorStoreIndex.from_vector_store(vector_store=vector_store)

In [5]:
# Sentence window VDB
chroma_collection_sentence_window = chroma_client.get_collection("ai_arxiv_sentence_window")
vector_store_sentence_window = ChromaVectorStore(chroma_collection=chroma_collection_sentence_window)
index_sentence_window = VectorStoreIndex.from_vector_store(vector_store=vector_store_sentence_window)


In [6]:
# Document summary VDB
from llama_index.core import load_index_from_storage
from llama_index.core import StorageContext

In [7]:
# storage_context = StorageContext.from_defaults(persist_dir="Obelix")
storage_context = StorageContext.from_defaults(persist_dir="ai_arxiv_doc_summary")
doc_summary_index = load_index_from_storage(llm=llm,
                                              storage_context=storage_context,
                                             embed_model = embed_model)

In [8]:
# Prompt template
with open("resources/text_qa_template.txt", 'r', encoding='utf-8') as file:
    text_qa_template_str = file.read()

text_qa_template = PromptTemplate(text_qa_template_str)

In [9]:
# Tonic Validate setup
benchmark = validate_api.get_benchmark(tonic_validate_benchmark_key)
scorer = ValidateScorer(metrics=[RetrievalPrecisionMetric(), AnswerSimilarityMetric()],
                        model_evaluator="gpt-3.5-turbo")

### Define query engines -------------------------------------------------------------------------------------------------


In [10]:
# Naive RAG
query_engine_naive = index.as_query_engine(llm = llm,
                                           text_qa_template=text_qa_template,
                                           similarity_top_k=3,
                                           embed_model=embed_model)

In [15]:
# Cohere Rerank
cohere_rerank = CohereRerank(api_key=config['cohere_api_key'], top_n=3)  # Ensure top_n matches k in naive RAG for comparability
query_engine_rerank = index.as_query_engine(
    similarity_top_k=10,
    text_qa_template=text_qa_template,
    node_postprocessors=[cohere_rerank],
    llm = llm,
    embed_model=embed_model
)

In [16]:
# HyDE
hyde = HyDEQueryTransform(include_original=True)
query_engine_hyde = TransformQueryEngine(query_engine_naive, hyde)

In [17]:
# HyDE + Cohere Rerank
query_engine_hyde_rerank = TransformQueryEngine(query_engine_rerank, hyde)

In [18]:
# Maximal Marginal Relevance (MMR)
query_engine_mmr = index.as_query_engine(vector_store_query_mode="mmr",
                                         similarity_top_k=3,
                                          embed_model=embed_model,
                                          llm=llm)


In [20]:
# Multi Query
vector_retriever = index.as_retriever(similarity_top_k=3)
retriever_multi_query = QueryFusionRetriever(
    [vector_retriever],
    similarity_top_k=3,
    num_queries=5,
    llm=llm,
    mode="reciprocal_rerank",
    use_async=False,
    verbose=True
)
query_engine_multi_query = RetrieverQueryEngine.from_args(retriever_multi_query,
                                                          verbose=True,
                                                          embed_model=embed_model,
                                                          llm=llm)

In [21]:
# Multi Query + Cohere rerank + simple fusion
retriever_multi_query_rerank = QueryFusionRetriever(
    [vector_retriever],
    similarity_top_k=10,
    llm=llm,
    num_queries=5,
    mode="simple",
    use_async=False,
    verbose=True
)
query_engine_multi_query_rerank = RetrieverQueryEngine.from_args(retriever_multi_query_rerank,
                                                                 verbose=True,
                                                                 node_postprocessors=[cohere_rerank],
                                                                 embed_model=embed_model,
                                                                 llm=llm)

In [22]:
## LLM Rerank
llm_rerank = LLMRerank(choice_batch_size=10, top_n=3)
query_engine_llm_rerank = index.as_query_engine(
    similarity_top_k=10,
    text_qa_template=text_qa_template,
    node_postprocessors=[llm_rerank],
    embed_model=embed_model,
    llm=llm
)

In [23]:
# HyDE + LLM Rerank
query_engine_hyde_llm_rerank = TransformQueryEngine(query_engine_llm_rerank, hyde)


In [24]:
# Sentence window retrieval
query_engine_sentence_window = index_sentence_window.as_query_engine(text_qa_template=text_qa_template,
                                                                     similarity_top_k=3,
                                                                      embed_model=embed_model,
                                                                      llm=llm)


In [28]:
# Sentence window retrieval + Cohere rerank
query_engine_sentence_window_rerank = index_sentence_window.as_query_engine(
    similarity_top_k=10,
    text_qa_template=text_qa_template,
    node_postprocessors=[cohere_rerank],
    embed_model=embed_model,
    llm=llm
)


In [25]:
# Sentence window retrieval + LLM Rerank
query_engine_sentence_window_llm_rerank = index_sentence_window.as_query_engine(
    similarity_top_k=10,
    text_qa_template=text_qa_template,
    node_postprocessors=[llm_rerank],
    embed_model=embed_model,
    llm=llm)

In [26]:
# Sentence window retrieval + HyDE
query_engine_sentence_window_hyde = TransformQueryEngine(query_engine_sentence_window, hyde)

In [29]:
# Sentence window retrieval + HyDE + Cohere Rerank
query_engine_sentence_window_hyde_rerank = TransformQueryEngine(query_engine_sentence_window_rerank, hyde)


In [30]:
# Sentence window retrieval + HyDE + LLM Rerank
query_engine_sentence_window_hyde_llm_rerank = TransformQueryEngine(query_engine_sentence_window_llm_rerank, hyde)


In [31]:
# Document summary index + Cohere Rerank
query_engine_doc_summary_rerank = doc_summary_index.as_query_engine(
    similarity_top_k=5,
    text_qa_template=text_qa_template,
    node_postprocessors=[cohere_rerank],
    llm=llm)


In [32]:
# Document summary index + HyDE + Cohere Rerank
query_engine_hyde_doc_summary_rerank = TransformQueryEngine(query_engine_doc_summary_rerank, hyde)


In [33]:
## Run experiments -------------------------------------------------------------------------------------------------------
# Dictionary of experiments, now referencing the predefined query engine objects
experiments = {
    # "Classic VDB + Naive RAG": query_engine_naive,
    # "Classic VDB + Cohere Rerank": query_engine_rerank,
    # "Classic VDB + LLM Rerank": query_engine_llm_rerank,
    # "Classic VDB + HyDE": query_engine_hyde,
    # "Classic VDB + HyDE + Cohere Rerank": query_engine_hyde_rerank,
    # "Classic VDB + HyDE + LLM Rerank": query_engine_hyde_llm_rerank,
    # "Classic VDB + Maximal Marginal Relevance (MMR)": query_engine_mmr,
    # "Classic VDB + Multi Query + Reciprocal": query_engine_multi_query,
    # "Classic VDB + Multi Query + Cohere rerank": query_engine_multi_query_rerank,
    # "Sentence window retrieval": query_engine_sentence_window,
    # "Sentence window retrieval + Cohere rerank": query_engine_sentence_window_rerank,
    # "Sentence window retrieval + LLM Rerank": query_engine_sentence_window_llm_rerank,
    # "Sentence window retrieval + HyDE": query_engine_sentence_window_hyde,
    # "Sentence window retrieval + HyDE + Cohere Rerank": query_engine_sentence_window_hyde_rerank,
    "Sentence window retrieval + HyDE + LLM Rerank": query_engine_sentence_window_hyde_llm_rerank,
    # "Document summary index + Cohere Rerank": query_engine_doc_summary_rerank,
    # "Document summary index + HyDE + Cohere Rerank": query_engine_hyde_doc_summary_rerank
}

In [34]:
# Initialize an empty DataFrame to collect results from all experiments
all_experiments_results_df = pd.DataFrame(columns=['Run', 'Experiment', 'OverallScores'])


In [35]:
# Loop through each experiment configuration, run it, and collect results
for experiment_name, query_engine in experiments.items():
    experiment_results_df = run_experiment(experiment_name,
                                            query_engine,
                                            scorer,
                                            benchmark,
                                            validate_api,
                                            config['tonic_validate_project_key'],
                                            upload_results=True,
                                            runs=10)  # Adjust the number of runs as needed

    # Append the results of this experiment to the master DataFrame
    all_experiments_results_df = pd.concat([all_experiments_results_df, experiment_results_df], ignore_index=True)


Retrieving responses: 0it [00:00, ?it/s]
Scoring responses: 0it [00:00, ?it/s]


Sentence window retrieval + HyDE + LLM Rerank Run 1 Overall Scores: {}


HTTPError: 422 Client Error: Unprocessable Entity for url: https://validate.tonic.ai/api/v1/projects/opycxzKWmtHosC25-r_Kd31EbfLHVBerNEBj675AeS4/runs/with_data